In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import os
import re
import time
import pandas
from collections import Counter

import nltk
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("omw-1.4")
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import wordpunct_tokenize
from nltk.tokenize import MWETokenizer


class InvertedIndex:
    """
    Construct Inverted Index
    """
    def __init__(self):
      self.inverted_index = None
      self.simpsonsTerms = None
      self.docIDS = None
        
        
    def read_data(self, path: str) -> list:
      """
      Read files from a directory and then append the data of each file into a list.
      """
      t = []
      self.simpsonsTerms = []
      self.docIDS = []

      for f in os.listdir(path):
        # if this is a text file
        if f[len(f)-3:] == "txt" and f != "examples.txt":
          file = open(path+"/"+f, 'r')
          # keep track of docID
          self.docIDS.append(f[:len(f)-4])
          txt = file.read()

          # trim document to everyting after external links
          l=txt.find("External links")
          if l == -1:
            # sometimes spelled differently
            l = txt.find("external links")
          txt = txt[l+len("External Links"):]
          t.append(txt)
          file.close()
        elif f[len(f)-3:] == "csv": # simpsons characters and locations
          self.simpsonsTerms.append(pandas.read_csv(path+"/"+f)["name"].tolist())
      for i in range(len(t)):
        # preprocess all of the wikipedia text
        t[i] = self.process_document(t[i])
      return t

    def process_document(self, document: str) -> list:
        """
        pre-process a document and return a list of its terms
        str->list"""
        #tokenize 
        document = wordpunct_tokenize(document)
        # construct dictionary of stopwords for O(1) lookup time when removing stopwords
        stops = Counter(stopwords.words())
        # list comp to casefold and remove non alpha chars from doc
        document = [re.sub("\W", "", w.lower()) for w in document if re.sub("\W", "", w.lower()) not in stops]
        return document

    
    def index_corpus(self, documents: list) -> None:
        """
        index given documents
        list->None"""
        # start clock
        start = time.time()

        """
        generate tokens for terms that are in the lists of characters and locations for simpsons
        have to normalize terms first usings regex and case folding
        as simpsons specific terms are sometimes multiple words, the MWE tokenizer flattens these to 
        single word tokens seperated by underscores, allowing us to continue the processing as we would 
        for single word tokenization
        """
        self.simpsonsTerms = [re.sub(r'[^A-Za-z0-9 ]+', "", t.lower()).split() for d in self.simpsonsTerms for t in d]
        tokenizer = MWETokenizer(self.simpsonsTerms)

        #construct list of tokens for the document
        doc_tokens = []
        for c,v in enumerate(documents):
          tokens = tokenizer.tokenize(v)
          # construct token list with each element in form [<token>, (<docID>, <position>)]
          tokens = [[token, (self.docIDS[c], count)] for count, token in enumerate(tokens) if token != ""]
          doc_tokens.append(tokens)

        # flatten list of tokens from a 2d list to a 1d list
        doc_tokens = [t for d in doc_tokens for t in d]
        """
        sort document tokens first by token alphabetically, then by document ID
        have to sort twice because 3.2 should come before 3.11 as the docID after the
        stop indicates episode number, so 3.2 < 3.11 by this ordering
        """
        doc_tokens.sort(key=lambda doc: (doc[0], int(doc[1][0][2:])))
        doc_tokens.sort(key=lambda doc: (doc[0], int(doc[1][0][0])))

        # make seperate copy of tokens to calculate document freq later
        doc_copy = doc_tokens.copy()
        
        # temp set to get list of unique index terms
        doc_set = list(dict.fromkeys([t[0] for t in doc_tokens]))
        self.inverted_index = []
        """
        iterate through each unique index term
        append to inverted index [<token>, [<docID, position>, ...]]
        shrink size of tokens each time for O(1) initial lookup-
        start at beginning of list each iteration, and go through tokens
        iteratively, adding to postings list, until the token changes.
        remove those processed tokens from token list, and start again from 
        beginning with new token
        """
        for i in range(len(doc_set)):
          self.inverted_index.append([doc_set[i],[doc_tokens[0][1]]])
          j = 1
          try:
            while doc_tokens[j][0] == self.inverted_index[i][0]:
              self.inverted_index[i][1].append(doc_tokens[j][1])          
              j += 1
            doc_tokens = doc_tokens[j:]
          except:
            pass
        # remove duplicate DOCIDS from postings list in document copy
        doc_copy = [[d[0], d[1][0]]for d in doc_copy]
        doc_copy = sorted(list(set([tuple(element) for element in doc_copy])))
        """
        calculate document frequency
        for each token in the inverted index, count each docID in
        postings list only once, and after inserting DF in inverted index
        try and except block for the final value in tokens list
        """
        for i in range(len(self.inverted_index)):
          count = 1
          value = doc_copy[0][0]
          j = 1
          try:
            while doc_copy[j][0] == doc_copy[0][0]:
              count += 1
              j += 1
          except:
            self.inverted_index[i].insert(1,count)
            continue
          doc_copy = doc_copy[j:]
          self.inverted_index[i].insert(1,count)


        # construct dictionary from current inverted index
        # structure: key: <token>, value: [ <DF>, [(<docID>, <position>), ... ] ]
        temp_dict = {}
        for term in self.inverted_index:
          temp_dict[term[0]] = [term[1], term[2]]
        self.inverted_index = temp_dict

        # stop time
        total_time = time.time() - start
        print("Total time to construct inverted index: " +str(total_time) + " seconds.")

        # calculate size of postings list
        items = 0
        for key in self.inverted_index:
          items += len(self.inverted_index[key][1])

        avg_len = items/len(self.inverted_index)

        print("Size of inverted index: \nunique terms indexed: " + str(len(self.inverted_index)))
        print("average postings list length: " +str(avg_len) + " postings")
        print("total unique positional indicies: " +str(items))
                         
    def dump(self, path: str, verbose = False) -> None:
        """
        provide a dump function to show index entries for a given set of terms        
        """
        # initialize multi word tokenizer for special simpsons terms
        tokenizer = MWETokenizer(self.simpsonsTerms)

        # read example terms and normalize them
        f = open(path, "r")
        examples = f.readlines()
        tokens = [example.lower().split() for example in examples]

        # tokenize the term, for each token search for matches in corpus
        for t in tokens:
          t = tokenizer.tokenize(t)
          for sub in t:
            print("*********************")
            print("Token: " +sub)
            try:
              print("Found in " + str(self.inverted_index[sub][0]) + " documents")
              if verbose == True:
                for post in self.inverted_index[sub][1]:
                  # if verbose is true, indicate each document and position for each finding
                  print("docID: " +post[0] +" @ position " +str(post[1]))
                print()
              else:
                # list each ID only once
                ids = list(dict.fromkeys([post[0] for post in self.inverted_index[sub][1]]))
                print("Found in documents: ")
                for id in ids:
                  print(id)
            except:
              print("TOKEN NOT FOUND IN INDEX\n")

       
    def proximity_search(self, term1: str, term2: str, window_size: int) -> dict:
        """
        This is Task 2"""
        #normalize input serach terms
        term1, term2 = re.sub(r'[^A-Za-z0-9 _]+', "", term1.lower()), re.sub(r'[^A-Za-z0-9 _]+', "", term2.lower())

        # get postings list
        term1_postings = self.inverted_index[term1]
        term2_postings = self.inverted_index[term2]

        #only search if both terms appear in corpus
        if term1_postings[0] != 0 and term2_postings[0] != 0:
          # remove DF for each postings list
          term1_postings, term2_postings = term1_postings[1], term2_postings[1]
          # and merge two lists based on docID
          term1_postings = [post for post in term1_postings if post[0] in [d[0] for d in term2_postings]]
          term2_postings = [post for post in term2_postings if post[0] in [d[0] for d in term1_postings]]
          # compute matches within window using list comprehension
          matches = [[p1,p2] for p1 in term1_postings for p2 in term2_postings if p1[0] == p2[0] and ((p1[1] in range(p2[1]-window_size, p2[1] + window_size) or (p2[1] in range(p1[1]-window_size, p1[1] + window_size))))]
          # construct dicitonary to return
          match_dict = {}
          
          for match in matches:
            match_dict[match[0][0]] = [match[0][1], match[1][1]]
          
          return match_dict
        # return empty dict if no matches found
        print("no matches found within windows for search terms!")
        return {}

        
        
    

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [8]:
def main():
    "main call function"
    index = InvertedIndex() # initilaise the index
    corpus = index.read_data('/content/drive/MyDrive/Colab Notebooks/Simpsons2022') # specify the directory path in which files are located
    index.index_corpus(corpus) # index documents/corpus
    
    return index
    
index = main()

Total time to construct inverted index: 15.814880609512329 seconds.
Size of inverted index: 
unique terms indexed: 13352
average postings list length: 6.995206710605153 postings
total unique positional indicies: 93400


In [9]:
# test dump functionality
index.dump("/content/drive/MyDrive/Colab Notebooks/Simpsons2022/examples.txt")

# perform proximity search on lisa and bart simpson with a window of 20 
matches = index.proximity_search("lisa","bart_simpson",20)
print(matches)

*********************
Token: bart
Found in 104 documents
Found in documents: 
3.1
3.2
3.3
3.4
3.5
3.6
3.7
3.8
3.9
3.10
3.11
3.12
3.13
3.14
3.15
3.16
3.18
3.19
3.20
3.21
3.22
3.23
3.24
4.1
4.5
4.6
4.7
4.8
4.9
4.10
4.11
4.12
4.13
4.14
4.15
4.16
4.18
4.19
4.20
4.21
4.22
5.1
5.2
5.3
5.4
5.5
5.6
5.7
5.8
5.9
5.10
5.11
5.12
5.13
5.14
5.15
5.16
5.17
5.18
5.19
5.20
5.21
5.22
6.1
6.2
6.3
6.4
6.5
6.6
6.7
6.8
6.9
6.10
6.14
6.16
6.17
6.18
6.19
6.20
6.21
6.22
6.23
6.24
6.25
7.1
7.2
7.3
7.4
7.5
7.6
7.7
7.8
7.9
7.10
7.11
7.12
7.13
7.15
7.18
7.20
7.21
7.22
7.24
7.25
*********************
Token: first
TOKEN NOT FOUND IN INDEX

*********************
Token: image
Found in 9 documents
Found in documents: 
3.7
3.9
3.16
3.18
5.13
7.2
7.5
7.21
7.25
*********************
Token: montage
Found in 7 documents
Found in documents: 
3.16
3.21
4.4
4.18
6.3
7.3
7.10
*********************
Token: well
TOKEN NOT FOUND IN INDEX

*********************
Token: top
Found in 51 documents
Found in documents: 
3.1
3.5
3.6
3.7
3.